## 2. Uczenie głębokie

In [23]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalAveragePooling2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

In [2]:
EPOCHS = 50
BATCH_SIZE = 64
STEPS_PER_EPOCH = 32
VALIDATION_STEPS = 64
LEARNING_RATE = 0.001
MODEL_FILE = 'models/vgg16-last-conv1.model'
##selected classses
classes = ["Lemon", "Raspberry", "Mandarine", "Pear 2", "Apple Red 1", "Strawberry", "Pomegranate", "Pepino", "Kaki", "Apricot", "Ginger Root", "Huckleberry"]

In [3]:
trdata_gen = ImageDataGenerator(1./255)
traindata = trdata_gen.flow_from_directory(directory="../data/Training",target_size=(224,224), batch_size=1,color_mode="rgb", classes=classes,class_mode="categorical") 

Found 9796 images belonging to 12 classes.


In [4]:
traindata.reset()
x_train=np.concatenate([traindata.next()[0] for i in range(traindata.__len__())])
y_train=np.concatenate([traindata.next()[1] for i in range(traindata.__len__())])
print(x_train.shape)
print(y_train.shape)

c:\users\julia\desktop\snr-20z-project\env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


(9796, 224, 224, 3)
(9796, 12)


In [5]:
tsdata_gen = ImageDataGenerator(1./255)
testdata = tsdata_gen.flow_from_directory(directory="../data/Test", target_size=(224,224), batch_size=1,color_mode="rgb", classes=classes,class_mode="categorical")

Found 1981 images belonging to 12 classes.


In [6]:
testdata.reset()
x_test=np.concatenate([testdata.next()[0] for i in range(testdata.__len__())])
y_test=np.concatenate([testdata.next()[1] for i in range(testdata.__len__())])
print(x_test.shape)
print(y_test.shape)

(1981, 224, 224, 3)
(1981, 12)


In [7]:
valdata_gen = ImageDataGenerator(1./255)
valdata = tsdata_gen.flow_from_directory(directory="../data/Validation", target_size=(224,224), batch_size=1,color_mode="rgb", classes=classes,class_mode="categorical")

Found 1735 images belonging to 12 classes.


In [8]:
valdata.reset()
x_val=np.concatenate([valdata.next()[0] for i in range(valdata.__len__())])
y_val=np.concatenate([valdata.next()[1] for i in range(valdata.__len__())])
print(x_val.shape)
print(y_val.shape)

(1735, 224, 224, 3)
(1735, 12)


In [9]:
CLASSES = len(traindata.class_indices)
# CLASSES = 131

In [10]:
base_model = VGG16(weights='imagenet', include_top = False , input_shape=(224,224, 3))
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
##freeze all  base_model layers and train the last ones
for layer in base_model.layers:
    layer.trainable = False

In [12]:
checkpoint = ModelCheckpoint("vgg16_conv_pretrained1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20, verbose=1, mode='auto')

2.1 Uczenie ostatniej warstwy splotowej i warstwy klasyfikującej

In [13]:
last_conv_flat = Flatten()(base_model.layers[-1].output)
# # add new classifier layers
class_dense_flat1 = Dense(4096, activation='relu')(last_conv_flat) 
class_dense_flat2 = Dense(4096, activation='relu')(class_dense_flat1) 
# -what about this
output = Dense(CLASSES, activation='softmax')(class_dense_flat2 )
# define new model
## maybe add dropout and maybe one more dense
last_conv_model = Model(inputs=base_model.inputs, outputs=output)
# summarize
last_conv_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
last_conv_model.get_layer('block5_conv3').trainable = True
last_conv_model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
print(len(traindata))

9796


In [25]:
opt = keras.optimizers.RMSprop(
    learning_rate=LEARNING_RATE,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name="RMSprop",
)

In [ ]:
last_conv_model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
last_conv_model_history = last_conv_model.fit(
    traindata,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(traindata)/BATCH_SIZE,
    validation_steps=len(valdata)//BATCH_SIZE,
    validation_data=valdata,
    shuffle=True,
    callbacks=[checkpoint, early_stopper]
)


model.save(MODEL_FILE)

Epoch 1/50
 39/153 [======>.......................] - ETA: 1:45 - loss: 1225.5162 - acc: 0.0142

In [ ]:
print(history.history.keys())
plt.plot(history_pretrained.history['acc'])
plt.plot(history_pretrained.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('acc_training_process_last_conv.png')
plt.show()

In [ ]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)
y_pred_val = model.predict(x_val)
print(sklearn.metrics.classification_report(y_train, y_pred_train))
print(sklearn.metrics.classification_report(y_test, y_pred_test)
print(sklearn.metrics.classification_report(y_val, y_pred_val))


2.2 Uczenie dwóch ostatnich warstw splotowych i warstwy klasyfikującej

In [20]:
two_last_conv_flat = Flatten()(base_model.layers[-1].output)
# # add new classifier layers
# class_dense_flat = Dense(1024, activation='relu')(last_conv_flat) -what about this
output = Dense(CLASSES, activation='softmax')(two_last_conv_flat)
# define new model
## maybe add dropout and maybe one more dense
two_last_conv_model = Model(inputs=base_model.inputs, outputs=output)
# summarize
two_last_conv_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [21]:
two_last_conv_model.get_layer('block5_conv3').trainable = True
two_last_conv_model.get_layer('block5_conv2').trainable = True
two_last_conv_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
two_last_conv_model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
two_last_conv_model_history = two_last_conv_model.fit(
    traindata,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=testdata,
    validation_steps=VALIDATION_STEPS,
    shuffle=True,
    callbacks=[checkpoint, early_stopper]
)


model.save(MODEL_FILE)

2.3 Uczenie całej sieci

In [ ]:
base_model = VGG16(weights='imagenet', include_top = True , input_shape=(224,224, 3))
print(base_model.summary())

2.4 Uproszczenie struktury